# The Economics of Sandwich Attacks
This is an analysis of sandwich attack economics on constant product market makers like Uniswap. I ultimately examine the optimal order of sandwich attackers if they were to cooperate.  

Extensions for the future:  
- Model other dexes (https://ethereum.stackexchange.com/a/107109)  
- If attackers are purely competitive, and the number of competitors is random with each spamming the block with bots to execute the trade, calculate the optimal number of bots (and gas) one should use to maximize their probability of success.


# Dependencies

In [15]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvas
from matplotlib.offsetbox import AnchoredText
from matplotlib.ticker import EngFormatter

import param

import panel as pn
import panel.widgets as pnw
pn.extension()

# import os
# from web3 import Web3, HTTPProvider

In [2]:
# # connect to node
# web3_var = os.environ.get('WEB3_PROVIDER_URI') # this environment var points to my node
# w3 = Web3(Web3.HTTPProvider(web3_var))
# w3.isConnected()

# Price Impact of a Single Trade
Let's first look at how a trade of A tokens in exchange for B tokens affects the price of B in the pool.  
Note that the fee is in decimals.

In [47]:
def swap(a_traded, a_reserves_i, b_reserves_i, fee):
    '''
    returns:
    - B tokens received in exchange for putting token A into LP
    - price impact on B (as a percent of b, expressed in decimals)
    - price impact on A
    '''
    a_traded = a_traded * (1 - fee)

    cp = a_reserves_i * b_reserves_i

    b_reserves_f = cp / (a_reserves_i + a_traded)
    b_received = b_reserves_i - b_reserves_f

    price_paid_per_b = a_traded / b_received
    price_impact_b = a_traded / (a_reserves_i + a_traded) # equals 1 - (a_reserves_i/b_reserves_i) / price_paid_per_b
    price_impact_a = b_received / (b_reserves_i + b_received)

    return b_received, price_impact_b, price_impact_a

def pn_plot(impacts):
    fig = Figure()
    fig.set_size_inches(12, 8)
    FigureCanvas(fig)
    ax = fig.add_subplot()
    ax.set_xlabel('A tokens input to the pool')
    impacts.plot(ax=ax)
    return fig

def px_impact_curve(a_reserve=1_000_000, b_reserve=1_000_000, fee=0.000, view_fn=pn_plot):
    a_ins = np.arange(1, a_reserve * 0.95, a_reserve/100) # create array of marginal A tokens to be input into the pool
    impacts = [swap(a, a_reserve, b_reserve, fee)[1] for a in a_ins]
    df_ = pd.DataFrame(data=impacts, index=a_ins, columns=['Price impact on B tokens'])
    return view_fn(df_)

float_slider = pn.widgets.FloatSlider(name='fee', format='1[.]000', start=0.00, end=0.010, step=0.001, value=0.00)
kw = dict(a_reserve=(1_000, 1_000_000), b_reserve=(1_000, 1_000_000), fee=float_slider) #(0.000, 0.1))
pn.interact(px_impact_curve, **kw)

BokehModel(combine_events=True, render_bundle={'docs_json': {'a6e95dcd-c554-47ce-b2f7-ac5f0abca127': {'defs': …

Column
    [0] Column
        [0] IntSlider(end=1000000, name='a_reserve', start=1000, value=1000000)
        [1] IntSlider(end=1000000, name='b_reserve', start=1000, value=1000000)
        [2] FloatSlider(end=0.01, format='1[.]000', name='fee', step=0.001)
    [1] Row
        [0] Matplotlib(Figure, name='interactive00551')

Notice how the price impact on `B` tokens does not depend on the `reserves of B` at all! Instead, it depends on the number of `A` tokens traded relative to the `reserves of A`, as well as the `fee`. 

# Single Sandwich Attack
Now let's look at how a single sandwich attack's return is impacted by different parameters of a pool.

In [6]:
def sandwich(reserves_a, reserves_b, fee, whale, a_tokens_invested):
    '''
    Returns: 
    - theoretical price impact of whale trade (without sandwich attack)
    - A tokens gained by sandwich attacker
    - % return of those A tokens gained
    - adverse impact on the whale trader
    '''
    # step 1: monitor whale trade; model price impact
    # whale trades A tokens for B tokens
    b_received_theo, price_impact_b_theo, price_impact_a_theo = swap(whale, reserves_a, reserves_b, fee)
    # print(f"Theoretical impact from whale trade: {price_impact_theoretical*100:,.3f}%")

    # step 2: model front run tx
    # a_tokens_invested is the sandwich attacker's initial investment
    b_received_front = swap(a_tokens_invested, reserves_a, reserves_b, fee)[0]
    reserves_a_after_front = reserves_a + a_tokens_invested
    reserves_b_after_front = reserves_b - b_received_front

    # step 3: model whale impact after front run tx
    b_received_whale = swap(whale, reserves_a_after_front, reserves_b_after_front, fee)[0]
    reserves_a_after_whale = reserves_a_after_front + whale
    reserves_b_after_whale = reserves_b_after_front - b_received_whale

    # step 4: model back run tx -- trading B tokens back to A tokens
    a_received_back = swap(b_received_front, reserves_b_after_whale, reserves_a_after_whale, fee)[0]
    reserves_a_after_back = reserves_a_after_whale - a_received_back
    reserves_b_after_back = reserves_b_after_whale + b_received_front

    # step 5: calculate profit
    a_tokens_gained = a_received_back - a_tokens_invested
    a_return = a_tokens_gained/a_tokens_invested
    # print(f'A tokens gained: {a_tokens_gained:,.2f}')
    # print(f'Percent return: {a_return:,.3f}%')

    # step 6: calculate max gas fee WTP (willingness to pay)
    fee_wtp = a_tokens_gained

    # step 7: calculate adverse impact on whale trade
    adverse_impact = b_received_whale - b_received_theo
    # print(f'Adverse impact on whale (in B tokens): {adverse_impact:,.2f}')

    # print(reserves_a_after_front, reserves_a_after_whale, reserves_a_after_back)

    return price_impact_b_theo, price_impact_a_theo, a_tokens_gained, a_return, adverse_impact

In [46]:
def single_sand_plot(returns):
    fig = Figure()
    fig.set_size_inches(12, 8)
    FigureCanvas(fig)
    ax = fig.add_subplot()
    ax.set_xlabel('A tokens traded by searcher')
    returns.plot(ax=ax)
    return fig

def single_sand_curve(a_reserve=1_000_000, b_reserve=1_000_000, whale=10_000, fee=0.0000, view_fn=single_sand_plot):
    a_ins = np.arange(10, a_reserve*0.95, 100).tolist()
    returns = [sandwich(a_reserve, b_reserve, fee, whale, a)[3] for a in a_ins]
    df_ = pd.DataFrame(data=returns, index=a_ins, columns=['Returns to searcher'])
    return view_fn(df_)

# set ranges of parameters
float_slider = pn.widgets.FloatSlider(name='fee', format='1[.]000', start=0.00, end=0.010, step=0.001, value=0.00)
kw = dict(a_reserve=(1_000, 1_000_000), b_reserve=(1_000, 1_000_000), whale=(100, 100_000), fee=float_slider)
pn.interact(single_sand_curve, **kw)

BokehModel(combine_events=True, render_bundle={'docs_json': {'171ef877-284c-4d90-b9d6-d068872b9013': {'defs': …

Column
    [0] Column
        [0] IntSlider(end=1000000, name='a_reserve', start=1000, value=1000000)
        [1] IntSlider(end=1000000, name='b_reserve', start=1000, value=1000000)
        [2] IntSlider(end=100000, name='whale', start=100, value=10000)
        [3] FloatSlider(end=0.01, format='1[.]000', name='fee', step=0.001)
    [1] Row
        [0] Matplotlib(Figure, name='interactive00540')

Note that the returns decline as the size of the attack increases. Also, the returns are not affected by the `reserves of B` at all! Returns are impacted by the `reserves of A`, the size of the attacker's trade, the size of the `whale trade`, and the `fee`.  

# Multiple Sandwich Attackers
Now let's derive the optimal order of multiple sandwich attacks on a single whale trade. The parameters `front_asc` and `back_asc` denote if the front- and back-running trades of the sandwich attacks are set in ascending order (by size). If set to `True` (or `1`), the smallest trades are placed first. 

In [8]:
def multi_sandwich(reserves_a, reserves_b, fee, whale, a_input, front_asc=True, back_asc=True):
    '''
    set front_asc and/or back_asc to True to put the smallest trades first for frontrun and backrun transactions
    '''
    a_received = [] # number of A tokens received by each trader (at the end of the sandwich attack)
    b_received = [] # number of B tokens received by each trader (at the beginnging of the sandwich attack)

    if front_asc==False: # front in descending order
        a_order = a_input[::-1]
    else:
        a_order = a_input

    # iterate through all front runs
    for a in a_order:
        b_received_front = swap(a, reserves_a, reserves_b, fee)[0]
        b_received.append(b_received_front)
        reserves_a = reserves_a + a # update reserves for A and B tokens
        reserves_b = reserves_b - b_received_front

    # whale trade
    b_received_whale = swap(whale, reserves_a, reserves_b, fee)[0]
    reserves_a = reserves_a + whale # update reserves for A and B tokens
    reserves_b = reserves_b - b_received_whale

    if front_asc != back_asc: # back_asc==False:
        b_received = b_received[::-1]

    # iterate through all back runs -- get A tokens back
    for b in b_received:
        a_received_back = swap(b, reserves_b, reserves_a, fee)[0]
        a_received.append(a_received_back)
        reserves_a = reserves_a - a_received_back
        reserves_b = reserves_b + b_received_front

    if back_asc==False: # adjust order of a_received if necessary
        a_received = a_received[::-1]

    returns = [a_out/a_in - 1 for a_out, a_in in zip(a_received, a_input)]
    mean_return = np.mean(returns)
    stdev = np.std(returns)

    return mean_return, stdev, returns

In [43]:
def multi_sand_plot(df_):
    fig = Figure()
    fig.set_size_inches(12, 8)
    FigureCanvas(fig)
    ax = fig.add_subplot()    
    ax.set_xlabel('Size of Sandwich Attack (in A Tokens)')
    ax.set_ylabel('Return to Searchers')

    returns = df_['returns to searchers']
        
    returns.plot.bar(ax=ax)

    anchored_text1 = AnchoredText(f"Average Return: {df_['returns to searchers'].mean():,.3%}", loc=2)
    anchored_text2 = AnchoredText(f"StDev of Returns: {df_['returns to searchers'].std():,.3%}", loc=1)
    ax.add_artist(anchored_text1)
    ax.add_artist(anchored_text2)

    ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=35, ha='right')

    plt.tight_layout()
    
    return fig

# assume size of attacks is normally distributed around, say, 2500
attack_sizes =  np.random.normal(loc=2500, scale=500, size=20) # normally distributed around 2500, stdev of 500, 20 values
attack_sizes = sorted([int(a) for a in attack_sizes])

def multi_sand_data(a_reserve=1_000_000, b_reserve=1_000_000, whale=10_000, fee=0.0000, front_asc=True, back_asc=True, view_fn=multi_sand_plot):
    # attack_sizes = np.arange(100, whale*0.50, (whale*0.50 - 100) / 10).tolist()
    # attack_sizes =  np.arange(500, 5000, 500).tolist()
    mean_return, std_return, returns = multi_sandwich(a_reserve, b_reserve, fee, whale, attack_sizes, front_asc, back_asc)
    df_ = pd.DataFrame(
        data = {"returns to searchers": returns, "mean return":[mean_return] * len(returns), "stdev return": [std_return] * len(returns)}, 
        index = attack_sizes)
    return view_fn(df_)

# set ranges of parameters
float_slider = pn.widgets.FloatSlider(name='fee', format='1[.]000', start=0.00, end=0.010, step=0.001, value=0.00)
kw = dict(a_reserve=(1_000, 100_0000), b_reserve=(1_000, 100_0000), whale=(100, 100_000), fee=float_slider, front_asc=(0, 1), back_asc=(0, 1))
pn.interact(multi_sand_data, **kw)

# class MultiSandwich(param.Parameterized):
#     FrontRun_Order = param.Selector(objects=[True, False])
#     BackRun_Order = param.Selector(objects=[True, False])
#     a_reserve=param.Integer(default=1_000_000, bounds=(1_000, 1_000_000))
#     b_reserve=param.Integer(default=1_000_000, bounds=(1_000, 1_000_000))
#     whale=param.Integer(default=100_000, bounds=(100, 1_000_000))
#     fee=param.Number(default=0.000, bounds=(0.000, 0.001))

#     def view(self):
#         return multi_sand_data(self.a_reserve, self.b_reserve, self.whale, self.fee, self.FrontRun_Order, self.BackRun_Order)

# obj = MultiSandwich()
# pn.Row(obj.param, obj.view)

BokehModel(combine_events=True, render_bundle={'docs_json': {'32a82368-8dd2-4805-82e3-6c6e558ca9a5': {'defs': …

Column
    [0] Column
        [0] IntSlider(end=1000000, name='a_reserve', start=1000, value=1000000)
        [1] IntSlider(end=1000000, name='b_reserve', start=1000, value=1000000)
        [2] IntSlider(end=100000, name='whale', start=100, value=10000)
        [3] FloatSlider(end=0.01, format='1[.]000', name='fee', step=0.001)
        [4] IntSlider(name='front_asc', value=True)
        [5] IntSlider(name='back_asc', value=True)
    [1] Row
        [0] Matplotlib(Figure, name='interactive00505')

<Figure size 432x288 with 0 Axes>

### Takeaways for multiple sandwich attacks on the same whale trade

- Not surprisingly, any given searcher will want to be first at the front-run and the back-run transactions. They benefit the most from the run-up in price due to multiple traders buying up token B, and then they are the first to exit the trade (thereby avoiding the price decline in token B driven by selling pressure).
- The larger the searcher is, the more they are negatively impacted by being placed later in the queue for both the front- and back-run transactions.
- If all searchers were to cooperate, the optimal solution for the group is usually to use a **descending order for the front-run trades, and an ascending order for the back-run trades**. This configuration leads to the highest or second highest average return for the group, and the lowest standard deviation of returns (i.e., a 'democratized' outcome).

# Sources:  
https://arxiv.org/pdf/2012.08040.pdf  
https://blog.coinfabrik.com/cryptocurrency/automated-market-making-mechanisms-and-issues-in-uniswap-balancer-and-curve/  
https://ethereum.stackexchange.com/questions/102063/understand-price-impact-and-liquidity-in-pancakeswap  
https://dailydefi.org/articles/price-impact-and-how-to-calculate/  